# XQ:Select CPU

Runtime -> Change Runtime type -> CPU

#XQ:Mount Drive

In [ ]:
# import packages
## for mount drive purpose
import os
from google.colab import drive

In [ ]:
# mount drive
drive.mount('/content/drive/', force_remount=True)
os.chdir('/content/drive/My Drive/Colab_Notebooks/Graph_transformer')

Mounted at /content/drive/


# Step 1 PPI graph embedding

In [ ]:
from utils import (get_graph_from_file, get_gene_idx_dict_from_file)

In [ ]:
input_folder = 'input'

edge_list_file_path = f'{input_folder}/interactom_edges.txt' # stores the edges for the largest connected component in human Interactome
node_file_path = f'{input_folder}/interactom_nodes.txt'   # stores the nodes for the largest connected component in human Interactome

## Embed graph with mapped ids.
Map the ids to the gene_ids first, so that graph nodes are 0, 1, ... number_of_nodes. Then, use this graph with new node ids to do the following tasks.

In [ ]:
import networkx as nx
from utils import (get_graph_from_file, get_gene_idx_dict_from_file)
import numpy as np
import scipy.sparse as sp

In [ ]:
def get_graph_from_file_and_map_ids(network_file, node_dict, **kwargs):
    """
        generate a graph based on the input file
        The input file is provided by Joerg Menche et al. in their paper's supplementary
        Thus modify their function to parse the file and get the graph
        The function returns:
        G: the graph with self loop removed
    """

    defaultKwargs = {'self_link': True}
    kwargs = { **defaultKwargs, **kwargs}

    G = nx.Graph()
    network_file = open(network_file,'r')
    for line in network_file:
        # lines starting with '#' will be ignored
        if line[0]=='#':
            continue
        line_data   = line.strip().split('\t')
        gene1 = line_data[0]
        gene2 = line_data[1]

        G.add_edge(node_dict[gene1],node_dict[gene2])

    # remove self links
    if not kwargs['self_link']:
        remove_self_links(G)
    return G
#------------------------------------------------------------------------------#
def remove_self_links(G):
    sl = nx.selfloop_edges(G)
    G.remove_edges_from(sl)

## Node2Vec

*   https://github.com/aditya-grover/node2vec
*   https://github.com/eliorc/node2vec
*   node2vec uses gensim word2vec: https://radimrehurek.com/gensim/models/word2vec.html

In [ ]:
!pip install node2vec

In [ ]:
from node2vec import Node2Vec

In [ ]:
def node2vec_embed(G_sub, dim):
  # use default setting from https://github.com/eliorc/node2vec
  node2vec = Node2Vec(G_sub, dimensions = dim, walk_length=30, num_walks=200, workers=4)
  # Embed nodes, use default setting from https://github.com/eliorc/node2vec
  model = node2vec.fit(window=2, min_count=1, batch_words=4)
  # get embedding matrix
  Z = model.wv.vectors
  return Z

In [ ]:
# get graph nodes
node_idx_dict = get_gene_idx_dict_from_file(node_file_path)
G_sub = get_graph_from_file_and_map_ids(edge_list_file_path, node_idx_dict)
Z = node2vec_embed(G_sub, 64)

Computing transition probabilities:   0%|          | 0/13329 [00:00<?, ?it/s]

### write Z to the file

In [ ]:
import pickle

In [ ]:
# write Z to the file
with open('embedding/node2nev_emb_64_for_PE', "wb") as fp:
    pickle.dump(Z, fp)

## LPE

In [ ]:
import numpy as np
from scipy import sparse as sp
from utils import get_gene_idx_dict_from_file
import networkx as nx

In [ ]:
edge_list_file_path = f'interactom_edges.txt' # stores the edges for the largest connected component in human Interactome
node_file_path = f'{input_folder}/interactom_nodes.txt'

In [ ]:
# get graph nodes
node_idx_dict = get_gene_idx_dict_from_file(node_file_path)
G_sub = get_graph_from_file_and_map_ids(edge_list_file_path, node_idx_dict)

In [ ]:
A = nx.adjacency_matrix(G_sub)  # sparse matrix, reordered the nodes so that is from small to large
degrees = [G_sub.degree[k] for k in range(G_sub.number_of_nodes())]
N = sp.diags(np.asarray(degrees) ** -0.5, dtype=float)
L = sp.eye(G_sub.number_of_nodes()) - N * A * N

In [ ]:
# Eigenvectors with numpy
EigVal, EigVec = np.linalg.eig(L.toarray())
idx = EigVal.argsort() # increasing order
EigVal, EigVec = EigVal[idx], np.real(EigVec[:,idx])

In [ ]:
from utils import matrix_to_file, array_to_file

In [ ]:
LPE = EigVec[:,0]

In [ ]:
pos_enc_dim = 64

In [ ]:
LPE = EigVec[:,:pos_enc_dim]

In [ ]:
LPE_file = "embedding/LPE.tsv"

In [ ]:
matrix_to_file(LPE, LPE_file)

## GEE

In [ ]:
input_folder = 'input'
RR = 'RR1'
node_file_path = f'{input_folder}/interactom_nodes.txt'   # stores the nodes for the largest connected component in human Interactome
train_file_path = f'{input_folder}/{RR}/train_set.tsv'
test_file_path = f'{input_folder}/{RR}/test_set.tsv'

In [ ]:
from utils import get_gene_idx_dict_from_file
import numpy as np

In [ ]:
def get_disease_dict(train_file_path, test_file_path):
    dis_id_dis_dict = {}     #{disease_id: disease_name}
    dis_dis_id_dict = {} #{disease_name: disease_id}
    dis_cnt_dict = {}  # {disease_id: disease_cnt}
    dis_gene_dict = {}  # {disease:[gene1,gene2,...]}

    id = 0
    for file_path in [train_file_path, test_file_path]:
      f = open(file_path, "r")
      head = True
      for line in f:
          if head:
              head = False
              continue

          row = line.strip().split("\t")
          dis_pair, disease_a_genes, disease_b_genes, all_genes, rr = row

          disease_a, disease_b = dis_pair.split("&")
          dis_gene_dict[disease_a] = disease_a_genes.split(",")
          dis_gene_dict[disease_b] = disease_b_genes.split(",")

          for dis in [disease_a, disease_b]:
            if dis not in dis_dis_id_dict:
              dis_dis_id_dict[dis] = id
              dis_id_dis_dict[id] = dis
              dis_dis_id_dict[id] = 1
              dis_cnt_dict[dis] = len(dis_gene_dict[dis]) #the disease in disease pairs in the train set and test set can repeat, only need to cnt the number of genes when first see it.
              id += 1



      f.close()

    return dis_id_dis_dict, dis_dis_id_dict, dis_cnt_dict, dis_gene_dict

def get_W(dis_cnt_dict, dis_gene_dict, dis_dis_id_dict, node_idx_dict):
  """
    dis_id_dis_dict = {}     #{disease_id: disease_name}
    dis_dis_id_dict = {} #{disease_name: disease_id}
    dis_cnt_dict = {}  # {disease_id: disease_cnt}
    dis_gene_dict = {}  # {disease:[gene1,gene2,...]}
    node_idx_dict: {gene: mapped_gene_id_from_0}
  """
  k = len(dis_cnt_dict.keys())
  n = len(node_idx_dict.keys())
  W = np.zeros((n,k))

  not_in = set()
  in_graph = set()
  for dis, gene_list in dis_gene_dict.items():
    dis_id = dis_dis_id_dict[dis]
    dis_cnt = dis_cnt_dict[dis]
    for gene in gene_list:
      if gene not in node_idx_dict:
        not_in.add(gene)
      else:
        gene_id = node_idx_dict[gene]
        W[gene_id,dis_id] = 1/dis_cnt
        in_graph.add(gene)
  # print(f'genes not in graph: {len(not_in)}')
  # print(f'genes in graph: {len(in_graph)}')
  # print(f'total genes: {len(not_in)+len(in_graph)} ')
  return W

In [ ]:
node_idx_dict = get_gene_idx_dict_from_file(node_file_path)

In [ ]:
dis_id_dis_dict, dis_dis_id_dict, dis_cnt_dict, dis_gene_dict = get_disease_dict(train_file_path, test_file_path)
W = get_W(dis_cnt_dict, dis_gene_dict, dis_dis_id_dict, node_idx_dict)

### GEE embed: Z = AW

In [ ]:
A = nx.adjacency_matrix(G_sub)
Z = A.dot(W)

In [ ]:
from scipy.linalg import svd
# take 64 dims of U
U, s, VT = svd(Z)
idx = s.argsort()[::-1] # decreasing order
s, U = s[idx], U[:,idx]

In [ ]:
pos_enc_dim = 64
Z_U = U[:,:pos_enc_dim]

#### write GEE matrix related to the file

In [ ]:
from utils import matrix_to_file, array_to_file

In [ ]:
GEE_file = f'embedding/{RR}/GEE_Z_U.tsv' # The file that contains the GEE weight matrix
matrix_to_file(Z_U, GEE_file)